In [ ]:
# 아마존 닷컴 분야별 베스트셀러 상품 크롤러

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time
import sys
import re
import math
import numpy 
import pandas as pd   
import random
import os
import pyautogui
import urllib.request
import urllib


# 학습목표 1 : 사용자에게 다양한 메뉴를 보여 준 후 카테고리값을 입력 받아 해당 카테고리 메뉴를 실행한다.
# Step 2. 사용자에게 카테고리 메뉴를 보여주고 정보를 입력 받습니다.
print("=" *80)
print("     아마존 닷컴의 분야별 Best Seller 상품 정보 추출하기")
print("=" *80)

sec = input('''
    1.Amazon Devices & Accessories     2.Amazon Launchpad            3.Appliances
    4.Apps & Games                     5.Arts, Crafts & Sewing       6.Audible Books & Originals
    7.Automotive                       8.Baby                        9.Beauty & Personal Care      
    10.Books                           11.Camera & Photo Products     12.CDs & Vinyl                             


    1.위 분야 중에서 자료를 수집할 분야의 번호를  선택하세요: ''')

cnt = int(input('    2.해당 분야에서 크롤링 할 건수는 몇건입니까?(1-100 건 사이 입력): '))
page_cnt = math.ceil( cnt / 50)

f_dir = input("    3.파일을 저장할 폴더명만 쓰세요(예:c:\\py_temp\\):")
if f_dir =='' :
    f_dir = "c:\\py_temp\\"
    
print("\n")

if sec == '1' :
      sec_name='Amazon Devices and Accessories'
elif sec =='2' :
      sec_name='Amazon Launchpad'
elif sec =='3' :
      sec_name='Appliances'
elif sec =='4' :
      sec_name='Apps and Games'
elif sec =='5' :
      sec_name='Arts and Crafts and Sewing'
elif sec =='6' :
      sec_name='Audible Books and Originals'        
elif sec =='7' :
      sec_name='Automotive'        
elif sec =='8' :
      sec_name='Baby'
elif sec =='9' :
      sec_name='Beauty and Personal Care'
elif sec =='10' :
      sec_name='Books'
elif sec =='11' :
      sec_name='Camera and Photo'
elif sec =='12' :
      sec_name='CDs and Vinyl'
        
if cnt > 30 :
      print("    요청 건수가 많아서 시간이 제법 소요되오니 잠시만 기다려 주세요~~")
else :
      print("    요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~")
      
#Step 3. 저장될 파일위치와 이름을 지정 한 후 크롬 드라이버를 실행하여 페이지를 엽니다
query_txt='아마존닷컴'
query_url='https://www.amazon.com/bestsellers?ld=NSGoogle'

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt+'-'+sec_name)
os.chdir(f_dir+s+'-'+query_txt+'-'+sec_name)

ff_dir=f_dir+s+'-'+query_txt+'-'+sec_name
ff_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.txt'
fc_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.csv'
fx_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.xls'

s_time = time.time( )

s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)
    
driver.get(query_url)
driver.maximize_window()

time.sleep(3)

# 분야별 더보기 버튼을 눌러 페이지를 엽니다
if sec == '1' :
      driver.find_element(By.LINK_TEXT, 'Amazon Devices & Accessories').click( )
elif sec == '2' :                    
      driver.find_element(By.LINK_TEXT,'Amazon Launchpad').click( )
elif sec == '3' :
      driver.find_element(By.LINK_TEXT,'Appliances').click( )
elif sec == '4' : 
      driver.find_element(By.LINK_TEXT,'Apps & Games').click( )
elif sec == '5' :
      driver.find_element(By.LINK_TEXT,'Arts, Crafts & Sewing').click( )
elif sec == '6' :
      driver.find_element(By.LINK_TEXT,'Audible Books & Originals').click( )
elif sec == '7' :
      driver.find_element(By.LINK_TEXT,'Automotive').click( )  
elif sec == '8' :
      driver.find_element(By.LINK_TEXT,'Baby').click( )
elif sec == '9' :
      driver.find_element(By.LINK_TEXT,'Beauty and Personal Care').click( )
elif sec == '10' : 
      driver.find_element(By.LINK_TEXT,'Books').click( )
elif sec == '11' :
      driver.find_element(By.LINK_TEXT,'Camera and Photo').click( )
elif sec == '12' :
      driver.find_element(By.LINK_TEXT,'CDs and Vinyl').click( )


time.sleep(1)

# 학습목표 2 : 해당 카테고리의 데이터를 수집합니다.
#Step 4. 화면을 스크롤해서 아래로 이동한 후 요청된 데이터를 수집합니다.

def scroll_down(driver):      
    driver.execute_script("window.scrollBy(0,1100);")
    time.sleep(4)

# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만듭니다
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

# 이미지 추출 코드 추가
img_src2=[]   # 이미지 URL 저장변수
file_no = 0

         
ranking2=[]
title3=[]
price2=[]
score2=[]
sat_count2=[]
store2=[]

count = 1

# # 이미지 저장용 폴더 생성하기
img_dir = ff_dir+"\\images"
os.makedirs(img_dir)
os.chdir(img_dir)

for a in range(1,page_cnt + 1) :
    print('%s 페이지의 내용을 추출합니다~~' %a)
    
    for cc in range(1,7) :
        scroll_down(driver)
        
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    content_list = soup.find_all('div', attrs = {'class':'a-column a-span12 a-text-center _cDEzb_grid-column_2hIsc'})
    
    for li in content_list:
        print('%s번째 내용을 추출합니다~~' %count)   
        
        # 이미지 저장하기
        try :
            photo = li.find('div','a-section a-spacing-mini _cDEzb_noop_3Xbw5').find('img')['src']
        except AttributeError :
            continue
            
        file_no += 1

        urllib.request.urlretrieve(photo,str(file_no)+'.jpg')
        time.sleep(1)

        if cnt == file_no :
            break
        
        #텍스트 내용 추가 시작
        f = open(ff_name, 'a',encoding='UTF-8')
        f.write("-----------------------------------------------------"+"\n")

        # 판매순위
        print("-" *70)
        try :
            ranking = li.find('span',class_='zg-bdg-text').get_text().replace("#","")
        except AttributeError :
            ranking = ''

        print("1.판매순위:",ranking)
        f.write('1.판매순위:'+ ranking + "\n")
        ranking2.append(ranking)

        #제품 이름 
        try :
            title1 = li.find('div','_cDEzb_p13n-sc-css-line-clamp-3_g3dy1').get_text().replace("\n","")
        except AttributeError :
            title1 = ''
        else :
            title2=title1.translate(bmp_map).replace("\n","") 
            
        print("2.제품이름:", title2.replace("\n",""))
        f.write('2.제품이름:'+ title2 + "\n")
        title3.append(title2.replace("\n",""))

         # 가격
        try :
            price = li.find('span','a-size-base a-color-price').get_text().replace("\n","")
        except  :
            price = li.find('span','p13n-sc-price').get_text().replace("\n","")

        print("3.가격:", price.replace("\n",""))
        f.write('3.가격:'+ price + "\n")
        price2.append(price.replace("\n",""))

        #상품평수
        try :
            sat_count = li.find('span','a-size-small').get_text().replace(",","")
        except  :
            sat_count='0'

        print('4.상품평 수:',sat_count)
        f.write('4.상품평 수:'+ sat_count + "\n")
        sat_count2.append(sat_count)

        #상품 별점 
        try :
            score = li.find('span','a-icon-alt').get_text()
        except AttributeError :
            score=' '

        print('5.평점:',score)
        f.write('5.평점:'+ score + "\n")
        score2.append(score)

        print("-" *70)

        f.close( )

        time.sleep(0.1)

        if count > cnt :
            break
            
        count += 1
    
    a += 1
    b = str(a)
    
    try :
        driver.find_element(By.CLASS_NAME,'a-normal').click()
    except :
        print('데이터 수집이 완료되었습니다')
        break
    
#Step 5. 검색 결과를 다양한 형태로 저장하기
              
amazon_best_seller = pd.DataFrame()
amazon_best_seller['판매순위']=ranking2
amazon_best_seller['제품소개']=pd.Series(title3)
amazon_best_seller['판매가격']=pd.Series(price2)
amazon_best_seller['상품평 갯수']=pd.Series(sat_count2)
amazon_best_seller['상품평점']=pd.Series(score2)


# csv 형태로 저장하기
amazon_best_seller.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
amazon_best_seller.to_excel(fx_name ,index=True , engine='openpyxl')

e_time = time.time( )
t_time = e_time - s_time

# Step 6. 요약 정보를 출력하기
print("\n") 
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)    

In [6]:
#엑셀 파일에 이미지 삽입하기

import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api  #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                    #파이썬 쉘을 관리자 권한으로 실행한 후 불러오기로 이 소스 실행하기
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(3).ColumnWidth = 30   #  이미지 가로 사이즈에 맞게 컬럼 크기 조정
row_cnt = cnt+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120  #  이미지 세로 사이즈에 맞게 로우 크기 조정

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,cnt+2) :
    col_name='C'+str(a)
    col_name2.append(col_name)

for b in range(1,cnt+1) :
    file_name=img_dir+'\\'+str(b)+'.jpg'
    file_name2.append(file_name)
      
for i in range(0,cnt) :
    rng = ws.Range(col_name2[i])
    image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
    excel.Visible=True
    excel.ActiveWorkbook.Save()
    
#driver.close()